In [1]:
# Install Environment dependencies
!pip install hugsvision
# DONT LOAD WHAT IS NOT NEEDED
#!pip install pyvips
#!sudo apt-get -y install lynx
#!sudo apt-get -y install w3m
#!sudo apt-get install desktop-file-utils

#import modules
import pandas as pd
import os
from pathlib import Path
from hugsvision.dataio.VisionDataset import VisionDataset
from torchvision.transforms import InterpolationMode
InterpolationMode.BICUBIC
import shutil

os.environ["WANDB_DISABLED"] = "true"

# ONLY IMPORT WHAT IS NEEDED
#import tensorflow as tf
#from PIL import Image
#import matplotlib.pyplot as plt
#import cv2 as cv
#import PIL
#import numpy as np
#import torch
#import glob
#from skimage import io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 379.4 kB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.8/512.8 kB 1.2 MB/s eta 0:00:00
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl size=370081 sha256=e2e5746651215f0121f3af0c27c1e7bb5a7c55a1cdd6d1b0412df2399f1822c1
  Stored in directory: /root/.cache/pip/wheels/a3/5f/fa/f011e578cc76e1fc5be8dce30b3eb9fd00f337e744b3bba59b
Successfully built pycocotools


/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [2]:
# What does this do?
#PIL.Image.MAX_IMAGE_PIXELS = None

In [3]:
#load dataset labels and info

df_train1 = pd.read_csv('../input/mayo-clinic-1024-jpg-part1/train_with_groups.csv')
df_train2 = pd.read_csv('../input/mayo-clinic-1024-jpg-part2-1/train_with_groups.csv')
df_train3 = pd.read_csv('../input/mayo-clinic-1024-jpg-part3/train_with_groups.csv')
df_train4 = pd.read_csv('../input/mayo-clinic-1024-jpg-part4-1/train_with_groups.csv')
df_train5 = pd.read_csv('../input/mayo-clinic-1024-jpg-part5-1/train_with_groups.csv')
df_train6 = pd.read_csv('../input/mayo-clinic-1024-jpg-part6/train_with_groups.csv')
df_train7 = pd.read_csv('../input/mayo-clinic-1024-jpg-part7-1/train_with_groups.csv')
df_train8 = pd.read_csv('../input/mayo-clinic-1024-jpg-part8/train_with_groups.csv')
df_train9 = pd.read_csv('../input/mayo-clinic-1024-jpg-part9/train_with_groups.csv')
df_train10 = pd.read_csv('../input/mayo-clinic-1024-jpg-part10/train_with_groups.csv')

df_train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_id    754 non-null    object 
 1   center_id   754 non-null    int64  
 2   patient_id  754 non-null    object 
 3   image_num   754 non-null    int64  
 4   label       754 non-null    object 
 5   file_size   754 non-null    float64
 6   group       754 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 41.4+ KB


In [4]:
# define relevant Paths
train_source_path = Path('../input/mayo-clinic-1024-jpg-part1/train')
train_dest_path = Path('../working/data')
ce_train_path = Path('../working/data/CE')
laa_train_path = Path('../working/data/LAA')

# clean working directory
if os.path.isdir(train_dest_path):
    shutil.rmtree(train_dest_path)
    os.makedirs(train_dest_path)
else:
    os.makedirs(train_dest_path)
    
# create one folder for each image class    
os.mkdir(ce_train_path)
os.mkdir(laa_train_path)

In [5]:
# only take first 1k images REMOVE IN PRODUCTION
for img in list(train_source_path.glob('./*.jpg'))[0:1000]:
    image_id = img.stem[:8]
    image_label = df_train1[df_train1['image_id'] == image_id].iloc[0]["label"]
    shutil.copyfile(img, train_dest_path / image_label / img.name)

In [6]:
train, test, id2label, label2id = VisionDataset.fromImageFolder('/kaggle/working/data/', balanced=False)

Split Datasets...
train_ds:  850
+---------+-----+-----+-------+
| Dataset | CE  | LAA | Total |
+---------+-----+-----+-------+
|  Train  | 604 | 246 |  850  |
|  Test   | 102 | 48  |  150  |
+---------+-----+-----+-------+


________________________________________________________

In [7]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification
huggingface_model = 'google/vit-base-patch16-224-in21k'

trainer = VisionClassifierTrainer(
    model_name   = "MayoClinicModel",
    train        =  train,
    test         =  test,
    output_dir   = "../working/out/",
    max_epochs   = 1,
    batch_size   = 32, # On RTX 2080 Ti
    lr       = 2e-5,
    fp16     = True,
    model = ViTForImageClassification.from_pretrained(
        huggingface_model,
        num_labels = len(label2id),
        label2id   = label2id,
        id2label   = id2label
    ),
    feature_extractor = ViTFeatureExtractor.from_pretrained(huggingface_model),
)

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Using amp half precision backend
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 850
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 27


Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,No log,0.600330


***** Running Evaluation *****
  Num examples = 150
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ../working/out/MAYOCLINICMODEL/1_2022-07-26-12-30-27/trainer/
Configuration saved in ../working/out/MAYOCLINICMODEL/1_2022-07-26-12-30-27/trainer/config.json
Model weights saved in ../working/out/MAYOCLINICMODEL/1_2022-07-26-12-30-27/trainer/pytorch_model.bin
Configuration saved in ../working/out/MAYOCLINICMODEL/1_2022-07-26-12-30-27/model/config.json
Model weights saved in ../working/out/MAYOCLINICMODEL/1_2022-07-26-12-30-27/model/pytorch_model.bin
Feature extractor saved in ../working/out/MAYOCLINICMODEL/1_2022-07-26-12-30-27/feature_extractor/preprocessor_config.json


Model saved at: ../working/out/MAYOCLINICMODEL/1_2022-07-26-12-30-27
